# Compare MWAS methods for sanity tests and troubleshooting

In this version, we use the full new SNP set with a selected window

# Pick the regions we will test

In [1]:
library(data.table)

# prev_hits <- fread("
# Chr     pos        old_z       old_p
# 11   38247902        -27.1535308285104       2.30024742330298e-162
# 2    47933357        18.3327793004811        4.53147699327216e-75
# 7    1987910         10.1255085321387        4.25759742738181e-24
# 7    1987896         10.1255085321387        4.25759742738189e-24
# 7    1987797         10.0241523897721        1.19379483108027e-23
# 7    1987778         10.0105851568856        1.36940148731912e-23
# 12   2194742         -10.0072634920486       1.41615523554955e-23
# ")


In [2]:
prev_hits <- fread("
Chr     pos
1    73274305
1    73418161
1    73418205
1    73418313
1    73419188
1    73419830
")

In [3]:
#df <- fread("09.5-OUT_matched_SNP_meth_cov_chunked_EXPANSE_a2.csv")

In [4]:
df <- fread("09-OUT_matched_SNP_meth_cov_a2.csv")

## Try original code

In [5]:
###### model: learn elastic net model on training data 
######---------Input: trainX, trainY
######---------Return: selected features and coefficents

# original
elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        data.frame(
                        cvm = min(cv.fit$cvm),
                        lambda = cv.fit$lambda.min,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
        yfit = glmnet(
        trainX,
        trainY,
        lambda = cv.opt$lambda,
        alpha = cv.opt$alpha
                )       
        idf <- coef(yfit)
        idx <- which(idf != 0)
        selectf <- data.frame(
                features = idf@Dimnames[[1]][idx], 
                coefs = idf [idx]
        )
}

# modified to use lambda 1se and appropriate cvm
elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    a <- 0.5
    search <- foreach(ai = a, .combine = rbind) %dopar% {
        set.seed(42)
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        print(paste0("Dim of trainX: ", dim(trainX)))
        print(paste0("Len of trainY: ", length(trainY)))
        coef_matrix <- as.matrix(coef(cv.fit))

        non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
        print("Coefficients when fitting: ")
        print(non_zero_coefs)

        data.frame(
                        cvm = cv.fit$cvm[cv.fit$lambda == cv.fit$lambda.1se],
                        lambda = cv.fit$lambda.1se,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
    set.seed(42)
    yfit <- glmnet(
    trainX,
    trainY,
    lambda = cv.opt$lambda,
    alpha = cv.opt$alpha)

    coef_matrix <- as.matrix(coef(yfit))

    non_zero_coefs <- coef_matrix[coef_matrix != 0, , drop = FALSE]
    print("Coefficients when optimal: ")
    print(non_zero_coefs)

    idf <- coef(yfit)
    idx <- which(idf != 0)
    selectf <- data.frame(
            features = idf@Dimnames[[1]][idx], 
            coefs = idf [idx]
    )
}

MWAS <- function(gwas, weight, geno){
        z <- gwas %*% weight
        z.cor <- cor(geno)
        se <- sqrt(weight %*%  z.cor %*%  weight)
        z <- z/se
        p=pnorm(abs(z),lower.tail=F)*2
        return(c(z, p))
}

In [6]:
df <- df[which(df$Chr == 1), ]

In [7]:
i <- 2

In [8]:
library("glmnet")
library("e1071")
library("doParallel")

set.seed(2018)
wind <- c(5000,10000)
# output directory
#outd <- "/dcl02/lieber/shan/shizhong/finemapping/GWAS/tags/scz3/mwas/chr22/1/"
outd <- "20-OUT_original_mwas_sanity_test/"

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



## Replace all old objects with new objects in same format

In [9]:
#load("p1.rda", verbose = TRUE)

In [10]:
#p[1:10, 1:10]

### Methylation data

In [11]:
suppressWarnings(library(bsseq))

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The

In [ ]:
# load data for mwas
# load("./rda/caudate_mwas_data_chr22.rda")
load(df$methylation_data[i])

p <- getMeth(BSobj2)


rownames(p) <- start(BSobj2)

sites_to_test <- which(start(BSobj2) >= (73418205 - 500) & start(BSobj2) <= (73418205 + 500))
p <- p[sites_to_test, ]

# candidate cg
cg <- as.numeric(rownames(p))

# regress out covariates
#load("covs_for_meqtl.rda")

### covariates

In [ ]:
covs <- fread(df$cov_file[i])
covs <- t(covs)
colnames(covs) <- covs[1, ]
covs <- covs[2:nrow(covs), ]
# transpose so we have same orientation as original code

### Regress methylation data over covariates

In [ ]:
BSobj2$brnum <- gsub("Br0", "Br", BSobj2$brnum)
colnames(covs) <- gsub("Br0", "Br", colnames(covs))

In [ ]:
mat <- match(BSobj2$brnum,colnames(covs)) 
covs <- t(covs[,mat])
p.residual=matrix(NA,dim(p)[1],dim(p)[2])

In [ ]:
rownames(covs)[is.na(covs[, 'genoPC1'])] <- BSobj2$brnum[is.na(covs[, 'genoPC1'])]

In [ ]:
colnames(p.residual) <- BSobj2$brnum

In [ ]:
covs <- as.data.frame(covs)
# Convert all columns except Dx and Sex from character to numeric
cols_to_convert <- setdiff(names(covs), c("Dx", "Sex"))

for (col in cols_to_convert) {
  covs[[col]] <- as.numeric(covs[[col]])
}

# Print the modified data frame to check the conversion
#print(dat)


In [ ]:
for (i in 1:dim(p)[1]) { # For each methylation site
    dat <- as.data.frame(cbind(y = p[i,], covs))
    
    # Check for rows with NAs (the ones for which we don't have covariate data)
    valid_rows <- complete.cases(dat)
    
    if (sum(valid_rows) > 0) {
        dat_valid <- dat[valid_rows,]
        model.res <- lm(y ~ ., data = dat_valid)
        
        # Store residuals in the corresponding positions
        p.residual[i, valid_rows] <- resid(model.res)
    }
}


# for(i in 1:dim(p)[1]){ # foro each methylation site
#         dat <- as.data.frame(cbind(p[i,],covs))
#         colnames(dat) <- c("y",paste0("x",1:ncol(covs)))
#         model.res <- lm(reformulate(paste0("x",1:ncol(covs)), "y"),dat)
#         p.residual[i,] = resid(model.res) 
# }

In [ ]:
snp.gwas2 <- NULL

In [ ]:
load("p1.rda")

In [ ]:
# min(snp.gwas2$pos_hg38)
# max(snp.gwas2$pos_hg38)

In [ ]:
# load("p1.rda")
# pos_we_got <- snp.gwas2$pos_hg38
# saveRDS(pos_we_got, "20-intermediate_positions_in_old_set.csv")

In [ ]:
#pos_we_got <- readRDS("20-intermediate_positions_in_old_set.csv")

In [ ]:
#pos_we_got

In [ ]:
# head(snp.gwas2)

### summary stats

In [ ]:
library(data.table)
library(CpGWAS)

In [ ]:
ss_path <- "/home/naglemi/mwas/gwas/gwas_stat_scz"

In [ ]:
snp.gwas2 <- fread(ss_path, skip = 1, header = FALSE)
colnames(snp.gwas2) <- strsplit(readLines(ss_path, n = 1), "\t")[[1]]

In [ ]:
snp.gwas2$z <- log(snp.gwas2$OR)/snp.gwas2$SE

In [ ]:
snp.gwas2 <- snp.gwas2[, c(2, 1, 3, 3, 8, 4, 5, 20, 11)]

In [ ]:
head(snp.gwas2, n = 1)

In [ ]:
colnames(snp.gwas2)[1:5] <- c("snp", "chr", "pos_hg38", "pos_hg38", "info")

In [ ]:
snp.gwas2 <- snp.gwas2[which(snp.gwas2$chr == 1 & snp.gwas2$pos_hg38 >= (73274305-10000) & snp.gwas2$pos_hg38 <= (73419830 + 10000)), ]

In [ ]:
dim(snp.gwas2)

In [ ]:
snp.gwas2 <- snp.gwas2[order(snp.gwas2$pos_hg38), ]

In [ ]:
head(snp.gwas2)

In [ ]:
#snp.gwas2 <- snp.gwas2[which(snp.gwas2$pos_hg38 %in% pos_we_got), ]

In [ ]:
dim(snp.gwas2)

In [ ]:
# built predition models
idx.ea <- BSobj2$race == "CAUC"

### SNPs in LIBD population

#### For reference, first load Shizhong's formatted SNPs on Chr7

In [ ]:
snp2_sorted <- snp2[, order(names(snp2))]

In [ ]:
colnames(snp2) <- gsub("Br0", "Br", colnames(snp2))

In [ ]:
snp2 <- snp2[, colnames(snp2) %in% colnames(p.residual)]

In [ ]:
dim(snp2)

In [ ]:
head(snp2)

In [ ]:
snp2_positions <- stringr::str_split_fixed(rownames(snp2), ":", 3)[, 2]

#### Now let's load ours on Chr1

In [ ]:
paths <- list(pvar_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/libd_chr1.pvar",
              pgen_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/libd_chr1.pgen",
              psam_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/libd_chr1.psam")

my_SNPs <- loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)

In [ ]:
snp_indices_of_interest <- which(my_SNPs$pvar_dt$POS >= 73274305-10000 & my_SNPs$pvar_dt$POS <= 73419830 + 10000)

In [ ]:
snp3 <- pgenlibr::ReadList(my_SNPs$pgen,
                        variant_subset = snp_indices_of_interest)
colnames(snp3) <- my_SNPs$pvar_dt$ID[snp_indices_of_interest]
rownames(snp3) <- my_SNPs$psam$`#IID`

In [ ]:
snp3[1:10, 1:10]

In [ ]:
map3 <- data.frame(POS = stringr::str_split_fixed(colnames(snp3), ":", 3)[, 2])

In [ ]:
snp3 <- t(snp3)

In [ ]:
# snp3 <- snp3[which(map3$POS %in% pos_we_got), ]

In [ ]:
map3 <- data.frame(POS = stringr::str_split_fixed(rownames(snp3), ":", 3)[, 2])

In [ ]:
dim(map3)

### SNPs in reference population

In [ ]:
#snp.1kg.eur2

In [ ]:
paths <- list(pvar_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.pvar",
              pgen_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.pgen",
              psam_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.psam")

my_SNPs <- loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)

In [ ]:
snp_indices_of_interest <- which(my_SNPs$pvar_dt$POS >= 73274305-10000 & my_SNPs$pvar_dt$POS <= 73419830 + 10000)

In [ ]:
snp.1kg.eur2 <- pgenlibr::ReadList(my_SNPs$pgen,
                        variant_subset = snp_indices_of_interest)
colnames(snp.1kg.eur2) <- my_SNPs$pvar_dt$ID[snp_indices_of_interest]
rownames(snp.1kg.eur2) <- my_SNPs$psam$`IID`

In [ ]:
dim(snp.1kg.eur2)

In [ ]:
map.1kg.eur2 <- my_SNPs$pvar_dt

In [ ]:
map.1kg.eur2 <- map.1kg.eur2[snp_indices_of_interest, ]

In [ ]:
dim(map.1kg.eur2)

In [ ]:
#map.1kg.eur2 <- data.frame(POS = stringr::str_split_fixed(colnames(snp.1kg.eur2), ":", 3)[, 2])

In [ ]:
snp.1kg.eur2 <- t(snp.1kg.eur2)

In [ ]:
# snp3 <- snp3[which(map3$POS %in% pos_we_got), ]

### Set window size and any other parameters

In [ ]:
wind <- 10000

Is 1se vs min for lambda the problem?

## Run for all

In [ ]:
head(map3)

In [ ]:
dim(map3)

In [ ]:
dim(snp3)

In [ ]:
p.residual <- p.residual[, order(colnames(p.residual))]

In [ ]:
dim(snp3)

In [ ]:
p.residual

In [ ]:
snp3[1:10, 1:10]

In [ ]:
snp3 <- snp3[, colnames(snp3) %in% colnames(p.residual)]

In [ ]:
snp3 <- snp3[, order(colnames(snp3))]

In [ ]:
dim(snp3)

In [ ]:
for(k in 1:length(wind)){
    models.ea <- c()
    models.all <- c()
    for(i in 1:length(cg)){
    #for(i in 1){
            cat(i,"\n")
            print(paste0("This cg is: ", cg[i]))
            range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
            range2 <- cg[i] + wind[k]
            idx <- map3$POS > range1 & map3$POS < range2
            # go to next cg if no snps within window
            if(sum(idx) <= 1){
                    next
            }
            geno <- snp3[idx,] # changed snp2 to snp3
            rownames(geno) <- map3$POS[idx]
            trainX <- t(geno)
            trainY <- p.residual[i,]
            fit <- elastic.net(trainX,trainY)
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next

            fit$cg <- cg[i]
            models.all <- rbind(models.all,fit)
            # EA only
            trainX <- trainX[idx.ea,]
            if(sum(apply(trainX,2,var)!=0) <= 1){
                    next
            }
            trainY <- trainY[idx.ea]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            if(nrow(fit) == 0) next
            fit$cg <- cg[i]
            models.ea <- rbind(models.ea,fit)
    }
}

In [ ]:
models.ea <- models.ea[models.ea[,1] != "(Intercept)",]
models.all <- models.all[models.all[,1] != "(Intercept)",]

In [ ]:
if(!dir.exists(outd)) dir.create(outd)

In [ ]:
head(models.all)

In [ ]:
dim(models.all)

In [ ]:
models.all

In [ ]:
head(snp.1kg.eur2)

In [ ]:
head(map.1kg.eur2)

In [ ]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0, nrow = length(cg2), ncol = 2)
cat("Dimensions of relevant objects:\n")
cat("models.all:", dim(models.all), "\n")
cat("snp.gwas2:", dim(snp.gwas2), "\n")
cat("map.1kg.eur2:", dim(map.1kg.eur2), "\n")
cat("snp.1kg.eur2:", dim(snp.1kg.eur2), "\n\n")

for (i in 1:length(cg2)) {
    pos <- models.all[models.all$cg == cg2[i], 1]
    gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
    weight <- models.all[models.all$cg == cg2[i], 2]
    match_indices <- match(pos, map.1kg.eur2$POS)
    
    cat("Iteration:", i, "\n")
    cat("Current CG:", cg2[i], "\n")
    cat("Positions:\n")
    print(head(pos))
    cat("GWAS Z-scores:\n")
    print(head(gwas))
    cat("Weights:\n")
    print(head(weight))
    cat("Matching Indices:\n")
    print(head(match_indices))
    
    tryCatch({
        if (any(is.na(match_indices))) stop("NA values found in match_indices")
        if (any(match_indices > nrow(snp.1kg.eur2))) stop("Out of bounds indices found")
    }, error = function(e) {
        cat("Error detected:", e$message, "\n")
        cat("Dimensions of relevant objects at error detection:\n")
        cat("models.all:", dim(models.all), "\n")
        cat("snp.gwas2:", dim(snp.gwas2), "\n")
        cat("map.1kg.eur2:", dim(map.1kg.eur2), "\n")
        cat("snp.1kg.eur2:", dim(snp.1kg.eur2), "\n")
        cat("Positions causing error:\n")
        print(pos)
        cat("Matching Indices causing error:\n")
        print(match_indices)
        stop("Stopping execution due to error.")
    })
    
    geno <- NULL
    tryCatch({
        geno <- snp.1kg.eur2[match_indices, ]
    }, error = function(e) {
        cat("Error accessing genotype data at iteration:", i, "\n")
        cat("Error message:", e$message, "\n")
        stop("Stopping execution due to error.")
    })
    
    cat("Genotype Data:\n")
    print(head(geno))
    tryCatch({
        mwas.all[i, ] <- MWAS(gwas, weight, t(geno))
        cat("MWAS Results (z, p):\n")
        print(mwas.all[i, ])
        cat("\n")
    }, error = function(e) {
        cat("Error performing MWAS at iteration:", i, "\n")
        cat("Error message:", e$message, "\n")
        stop("Stopping execution due to error.")
    })
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z", "p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0, nrow = length(cg2), ncol = 2)
cat("Dimensions of relevant objects:\n")
cat("models.ea:", dim(models.ea), "\n")
cat("snp.gwas2:", dim(snp.gwas2), "\n")
cat("map.1kg.eur2:", dim(map.1kg.eur2), "\n")
cat("snp.1kg.eur2:", dim(snp.1kg.eur2), "\n\n")

for (i in 1:length(cg2)) {
    pos <- models.ea[models.ea$cg == cg2[i], 1]
    gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
    weight <- models.ea[models.ea$cg == cg2[i], 2]
    match_indices <- match(pos, map.1kg.eur2$POS)
    
    cat("Iteration:", i, "\n")
    cat("Current CG:", cg2[i], "\n")
    cat("Positions:\n")
    print(head(pos))
    cat("GWAS Z-scores:\n")
    print(head(gwas))
    cat("Weights:\n")
    print(head(weight))
    cat("Matching Indices:\n")
    print(head(match_indices))
    
    tryCatch({
        if (any(is.na(match_indices))) stop("NA values found in match_indices")
        if (any(match_indices > nrow(snp.1kg.eur2))) stop("Out of bounds indices found")
    }, error = function(e) {
        cat("Error detected:", e$message, "\n")
        cat("Dimensions of relevant objects at error detection:\n")
        cat("models.ea:", dim(models.ea), "\n")
        cat("snp.gwas2:", dim(snp.gwas2), "\n")
        cat("map.1kg.eur2:", dim(map.1kg.eur2), "\n")
        cat("snp.1kg.eur2:", dim(snp.1kg.eur2), "\n")
        cat("Positions causing error:\n")
        print(pos)
        cat("Matching Indices causing error:\n")
        print(match_indices)
        stop("Stopping execution due to error.")
    })
    
    geno <- NULL
    tryCatch({
        geno <- snp.1kg.eur2[match_indices, ]
    }, error = function(e) {
        cat("Error accessing genotype data at iteration:", i, "\n")
        cat("Error message:", e$message, "\n")
        stop("Stopping execution due to error.")
    })
    
    cat("Genotype Data:\n")
    print(head(geno))
    tryCatch({
        mwas.ea[i, ] <- MWAS(gwas, weight, t(geno))
        cat("MWAS Results (z, p):\n")
        print(mwas.ea[i, ])
        cat("\n")
    }, error = function(e) {
        cat("Error performing MWAS at iteration:", i, "\n")
        cat("Error message:", e$message, "\n")
        stop("Stopping execution due to error.")
    })
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z", "p")


In [ ]:
mwas.ea

In [ ]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
    pos <- models.all[models.all$cg == cg2[i],1]
    
    gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
    weight <- models.all[models.all$cg == cg2[i],2]
    geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
    mwas.all[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z","p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
    pos <- models.ea[models.ea$cg == cg2[i],1]
    gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
    weight <- models.ea[models.ea$cg == cg2[i],2]
    geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
    mwas.ea[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z","p")

# output models and mwas results
outf <- paste0(outd,"/models-a8-covnew.all.wind.",wind[k])
write.csv(models.all,outf)
outf <- paste0(outd,"/models-a8-covnew.ea.wind.",wind[k])
write.csv(models.ea,outf)
outf <- paste0(outd,"/mwas-a8-covnew.all.wind.",wind[k])
write.csv(mwas.all,outf)
outf <- paste0(outd,"/mwas-a8-covnew.ea.wind.",wind[k])
write.csv(mwas.ea,outf)

In [ ]:
mwas.all

We get the same results with old SNPs,

## Compare with results from CpGWAS

In [ ]:
# results <- fread("16a9par-OUT_stage2_MWAS_scz.csv")

# results <- results[which(results$chr == 7 & results$pos >= 1987413 & results$pos <= 1988332), ]

# results <- results[which(results$population == "EA" & results$region == "caud"), ]

# head(results)